# **Analyzing the Impact of Policy Changes on Medicare Inpatient Hospital Program Payments Using T-Tests**

This analysis aims to assess whether a significant difference exists in Medicare inpatient hospital program payments before and after a policy change in 2019. Using historical data from 2015 to 2021, we perform an **independent t-test** to compare the total program payments across these two periods.  

1. **Pre-Policy Period:** 2015–2018  
2. **Post-Policy Period:** 2019–2021  

To ensure the validity of the t-test, we first conduct a **Shapiro-Wilk normality test** on both periods' data to check for normal distribution. Then, we apply a **Welch’s t-test** (assuming unequal variances) to determine if there is a statistically significant difference in program payments before and after the policy change.  


## **Interpretation:**  
- If the **p-value < 0.05**, we conclude that there is a significant difference in program payments between the two periods, suggesting that the policy change had an impact.  
  
- If the **p-value ≥ 0.05**, we conclude that there is no statistically significant difference, indicating that the policy change may not have affected program payments substantially.  

This statistical test helps in understanding the financial implications of healthcare policy changes and assists in data-driven decision-making.

## **Form Hypotheses:**
- Null Hypothesis (H0): There is no significant difference in the mean values before and after the policy change.
  
- Alternative Hypothesis (H1): There is a significant difference in the mean values before and after the policy change.

## **Collect data:**
I need to use an additional file to concatenate with the original dataset, which contains data from 2016 to 2021, while the extra file includes data from 2015. The dataset paths are specific to my local machine, so feel free to modify them according to your setup. Since the workbook contains multiple worksheets, we must also specify the exact worksheet we are working with.

## **Independent T-Test Formula:**

The t-statistic for two independent samples is calculated as:

$$
t = \frac{\bar{X_1} - \bar{X_2}}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}
$$

Where:
- $ \bar{X_1} $ = Mean of the first sample (pre-policy period)  
- $ \bar{X_2} $ = Mean of the second sample (post-policy period)  
- $ s_1^2 $ = Variance of the first sample  
- $ s_2^2 $ = Variance of the second sample  
- $ n_1 $ = Sample size of the first group  
- $ n_2 $ = Sample size of the second group 


In [17]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import shapiro

file_path = "/Users/DELL/Desktop/Projects/CMS Program Statistics - Medicare Inpatient Hospital/2021/CPS MDCR INPT 2021.xlsx"
extra_file = "/Users/DELL/Desktop/Projects/CMS Program Statistics - Medicare Inpatient Hospital/2021/CPS MDCR INPT 2020.xlsx"


## **Clean data:**
Here, we will clean and preprocess the data to ensure accuracy in our analysis. Note: We will focus exclusively on the **All Beneficiaries** group, as it provides the most comprehensive and reliable information related to hospital program payments and utilization. We also need to point out which 

In [18]:
# Read the sheet "MDCR INPT HOSP 1" from the Excel file
df_1 = pd.read_excel(file_path, sheet_name="MDCR INPT HOSP 1", header=3)
# Extract the relevant section
all_beneficiaries_index = df_1[df_1["Type of Entitlement and Calendar Year"] == "All Beneficiaries"].index

if not all_beneficiaries_index.empty:
    start_index = all_beneficiaries_index[0] + 1
    df_1_all_beneficiaries = df_1.iloc[start_index:start_index + 6].copy()  # <---- Use .copy()

    df_1_all_beneficiaries["Year"] = pd.to_numeric(df_1_all_beneficiaries["Type of Entitlement and Calendar Year"], errors='coerce')
    df_1_all_beneficiaries = df_1_all_beneficiaries.dropna(subset=["Year", "Total Discharges"])
    df_1_all_beneficiaries["Year"] = df_1_all_beneficiaries["Year"].astype(int)


    # Load extra dataset containing 2015
    df_extra = pd.read_excel(extra_file, sheet_name="MDCR INPT HOSP 1", header=4)
    df_extra = df_extra.dropna(axis=1, how="all")

    df_extra.columns = ["Type of Entitlement and Calendar Year", "Total Original Medicare Part A Enrollees", 
                        "Total Persons With Utilization", "Total Discharges"] + [f"Unnamed_{i}" for i in range(len(df_extra.columns)-4)]

    df_extra = df_extra.dropna(how="all")

    extra_beneficiaries_index = df_extra[df_extra["Type of Entitlement and Calendar Year"] == "All Beneficiaries"].index

    if not extra_beneficiaries_index.empty:
        extra_start_index = extra_beneficiaries_index[0] + 1
        df_extra_all_beneficiaries = df_extra.iloc[extra_start_index:extra_start_index + 1].copy()  # <---- Use .copy()

        df_extra_all_beneficiaries["Year"] = pd.to_numeric(df_extra_all_beneficiaries["Type of Entitlement and Calendar Year"], errors='coerce')
        df_extra_all_beneficiaries = df_extra_all_beneficiaries.dropna(subset=["Year", "Total Discharges"])
        df_extra_all_beneficiaries["Year"] = df_extra_all_beneficiaries["Year"].astype(int)


        # Concatenate both datasets
        df_final = pd.concat([df_1_all_beneficiaries, df_extra_all_beneficiaries])

        # Sort by Year
        df_final = df_final.sort_values(by="Year")


We need to clearly define the pre-policy and post-policy periods again, using "Total Discharges" as the key variable for analysis. 

## **Shapiro-Wilk Test**
The Shapiro-Wilk test is a statistical test used to determine whether a given sample comes from a normally distributed population. It tests the null hypothesis that the data follows a normal distribution.

The test returns a test statistic (W) and a p-value. If the p-value is greater than a chosen significance level (typically 0.05), we fail to reject the null hypothesis, suggesting the data is normally distributed. If the p-value is less than the significance level, we reject the null hypothesis, suggesting the data is not normally distributed.

### **Formula for the Shapiro-Wilk Test:**
The Shapiro-Wilk test statistic (W) is given by

$$ W = \frac{\left( \sum_{i=1}^{n} a_i x_i \right)^2}{\sum_{i=1}^{n} (x_i - \bar{x})^2} $$

Where:
- $W$ = Shapiro-Wilk test statistic (used to test the normality of the sample data).
- $n$ = Number of data points in the sample.
- $a_i$ = The coefficients derived from the expected values of order statistics of a normally distributed sample.
- $x_i$ = The ordered sample values.
- $\bar{x}$ = The sample mean, calculated as $\bar{x} = \frac{1}{n} \sum_{i=1}^{n} x_i$.

This formula is used to test the null hypothesis that the sample data comes from a normally distributed population. The closer the value of $W$ is to 1, the more likely the data is normally distributed


In [19]:
# Ensure you're working with the correct dataframe (df_final) for pre-policy and post-policy
pre_policy = df_final[df_final['Year'] < 2019]
post_policy = df_final[df_final['Year'] >= 2019]

# Perform T-test for the 'Program Payment' variable (change to the appropriate column name, e.g., "Total Program Payments")
pre_policy_values = pre_policy['Total Discharges']  # Change to the column you're analyzing, e.g., 'Program Payment'
post_policy_values = post_policy['Total Discharges']  # Change to the column you're analyzing, e.g., 'Program Payment'

# Check for normality first (optional, but recommended)
_, p_pre = shapiro(pre_policy_values)
_, p_post = shapiro(post_policy_values)

print(f"Pre-policy normality p-value: {p_pre}")
print(f"Post-policy normality p-value: {p_post}")


Pre-policy normality p-value: 0.15059263497306208
Post-policy normality p-value: 0.365028744979078


# **Conduct T-Test:**

In [20]:
# Perform the T-test
t_stat, p_value = stats.ttest_ind(pre_policy_values, post_policy_values, equal_var=False)
# Output the result
print(p_value)
if p_value < 0.05:
    print("There is a significant difference in total discharges before and after the policy change.")
else:
    print("There is no significant difference in total discharges before and after the policy change.")

0.10689281796953051
There is no significant difference in total discharges before and after the policy change.


Since the T-test result indicated no significant difference, it suggests that the total discharges before and after the policy change did not differ in a statistically meaningful way at the chosen significance level (typically 0.05). Based on this test, it appears that the policy change may not have had a substantial impact on total discharges.

Although a sharp drop in total discharges was noticed from 2019 to 2020, the T-test provides important context. It compares the mean total discharges across the entire pre-policy period (2015–2018) and the post-policy period (2019–2021). The sharp drop in 2020 may be attributed to factors that the T-test doesn't account for, such as:

1. **Anomalies or Outliers:** The sharp decline in discharges in 2020 could be due to extraordinary external factors, like the COVID-19 pandemic, which caused significant disruptions in hospital operations. This kind of event could drastically affect discharges in a single year, making it an outlier.

2. **Yearly Variation:** The T-test examines overall trends and does not isolate yearly fluctuations. Even if 2020 is an outlier, its effect might not be significant enough to alter the overall results, as it is just one year within a larger multi-year comparison.

3. **Visual Trends vs. Statistical Significance:** A line graph might highlight sharp changes, like the one observed in 2020, but the T-test focuses on the overall distribution of data. A single year's anomaly may not be enough to influence the mean significantly when averaged with other years.